In [1]:
#
#### IMPORT LIBRARIES
#
# 
import os
import re
import json
import requests
from IPython.display import display
from neo4j import GraphDatabase, basic_auth

import numpy as np
import pandas as pd
import bs4
from bs4 import BeautifulSoup

import spacy
from spacy import displacy
from spacy.matcher import Matcher 
from spacy.tokens import Span 

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag, ne_chunk, dependencygraph
from nltk import RegexpParser
from nltk import Tree

import networkx as nx

# from tqdm.notebook import tqdm
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)


pd.set_option ('display.max_colwidth', None)
pd.set_option ('display.max_columns', None)

nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('brown')
nltk.download ('punkt')
nltk.download ('stopwords')
nltk.download ('wordnet')
import warnings

nlp = spacy.load('en_core_web_sm')

warnings.filterwarnings("ignore", category=UserWarning, module='bs4')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kunmi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/kunmi/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/kunmi/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package brown to /Users/kunmi/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kunmi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/kunmi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kunmi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/kunmi/miniforge3/lib/python3.9/site-pa

In [2]:
df = pd.read_excel ('../Data/preprocessed/NG_ELECTION_TWEETS_LABELLED.xlsx')

In [3]:
df.head (2)

,Sentiment,text
0,Positive,"RT @AtedoPeterside: Nobody ""says"" it better than our GoAmbassador @mrmacaronii. Take matters into your own hands by acquiring your PVC now.…"
1,Neutral,RT @1dernet: Getting Our PVC And Registering To Vote Come 2023 Is The Nonviolent Part To Take.\n#GetYourPVC \n@DeleFarotimi \n@Tsngcampaign ht…


In [4]:
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [5]:
def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [
    {'DEP':'ROOT'}, 
    {'DEP':'prep','OP':"?"},
    {'DEP':'agent','OP':"?"},  
    {'POS':'ADJ','OP':"?"}
  ] 

  matcher.add("matching_1", [pattern]) 

  matches = matcher (doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

In [6]:
def custom_cleaner (tweet):
  soup    = BeautifulSoup (tweet, 'lxml') # removing HTML entities such as '&amp', '&quot', '&gt'; lxml is the html parser
  souped  = soup.get_text ()
  link_re = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
  re_ = re.sub (link_re, "", souped)
  re_ = re.sub (r"#\S*", "", re_)       # Replace all hashtags with an empty string
  re_ = re.sub ("RT @\S*", " ", re_)
  re_ = re.sub (r"[^A-Za-z]+", " ", re_) # sustituting any non-alphabetic character that repeats one or more time

  tokens     = nltk.word_tokenize (re_)
  lower_case = [t.lower () for t in tokens]

  stop_words      = set (stopwords.words ('english'))
  filtered_result = list (filter (lambda l: l not in stop_words, lower_case))

  wordnet_lemmatizer = WordNetLemmatizer ()
  lemmas             = [wordnet_lemmatizer.lemmatize (t) for t in filtered_result]

  return lower_case

In [7]:
tqdm.pandas ()

for row, col  in tqdm(df['text'].iteritems (), total=df.shape[0]):
  df.loc[row, 'cleaned_tweet'] =  ' '.join (custom_cleaner (col))

100%|██████████| 11659/11659 [00:10<00:00, 1130.54it/s]


In [8]:
df.head (3)

,Sentiment,text,cleaned_tweet
0,Positive,"RT @AtedoPeterside: Nobody ""says"" it better than our GoAmbassador @mrmacaronii. Take matters into your own hands by acquiring your PVC now.…",nobody says it better than our goambassador mrmacaronii take matters into your own hands by acquiring your pvc now
1,Neutral,RT @1dernet: Getting Our PVC And Registering To Vote Come 2023 Is The Nonviolent Part To Take.\n#GetYourPVC \n@DeleFarotimi \n@Tsngcampaign ht…,getting our pvc and registering to vote come is the nonviolent part to take delefarotimi tsngcampaign ht
2,Neutral,RT @Chude__: Hear what late Dora Akunyeli said about Peter Obi.. \n\n#GetYourPVC\n#Competense2023\n#peterObi2023 https://t.co/L0u1FqY4dB,hear what late dora akunyeli said about peter obi


In [9]:
# entities = [get_relation(i) for i in tqdm(df['text'])]
reps = []
for text in df ['cleaned_tweet'] [0:3]:
  sentences = nltk.sent_tokenize(text)
  rep = []
  for sentence in sentences:
    try: 
      rep.append ({'entities': get_entities (sentence), 'relation': get_relation (sentence)})
    except (IndexError):
      pass

  reps.append (rep)


In [11]:
# entities = [get_relation(i) for i in tqdm(df['text'])]
reps = []

for text in tqdm (df ['cleaned_tweet'], total=df.shape[0]):
  # sentences = nltk.sent_tokenize(text)
  rep = {'entities': [], 'relation': []}
  # for sentence in sentences:
  try: 
    # {'entities': [*rep['entities'], *get_entities (sentence)], 'relation': [*rep['relation'], get_relation (sentence)]}
    rep = {'entities': get_entities (text), 'relation': get_relation (text)}
  except (IndexError):
    pass

  reps.append (rep)

100%|██████████| 11659/11659 [02:58<00:00, 65.49it/s]


In [12]:
# pip3 install neo4j-driver
# python3 example.py
driver = GraphDatabase.driver(
  "bolt://localhost:7687",
  auth=basic_auth("neo4j", password="12345"))

cypher_query = '''
MERGE (n:Subject {name:$word_1})
  ON CREATE SET n.count = 1
  ON MATCH SET n.count = n.count + 1  
MERGE (m:Object {name:$word_2})
  ON CREATE SET m.count = 1
  ON MATCH SET m.count = m.count + 1
MERGE (n) - [r:VERB {name: $verb}] -> (m)
  ON CREATE SET r.count = 1
  ON MATCH SET r.count = r.count + 1
''' 

with driver.session (database="neo4j") as session:

  for i in tqdm (range (len (reps)), total=len (reps)):
    rel = reps[i]['relation']
    try:
      session.write_transaction (
        lambda tx: tx.run (
          cypher_query,
          parameters={"word_1": reps[i]['entities'][0], "word_2": reps[i]['entities'][1], 'verb':rel})
      )

    except:
      pass
  
driver.close()

100%|██████████| 11659/11659 [06:07<00:00, 31.70it/s]
